In [1]:
using StatsBase
using Combinatorics
include("jl/sums.jl");

In [2]:
# toy data

Z = [1 1 2 2 3 3 4 4 4 5 5] # group partition
D = [3 4 2 5 6 4 3 2 5 2 2]; # degree sequence

First thing we'll do: let's check the formula

$$\sum_{R \in [n]^\ell} \mathbb{I}(\#(\mathbf{z}_R) = p) \sigma(\theta_R) = \sum_{y: \#y = p}\prod_{a = 1}^{\ell}v_{y_a}\;,$$

In this formula, we're allowing R to range over $[n]^\ell$, where $\ell$ is the number of nodes per hyperedge, and $n$ the number of nodes.  $p$ is a specified permutation. This is **different** from the current version of the nodes, and matches the conversation we had over email. 

The function `evalSumNaive` performs the summation on the lefthand side, while the function `evalSumPV` uses the product-of-volumes representation on the righthand side. Both of these functions are defined in `jl/sums.jl`.



In [3]:
# test: check that these two functions give the same result on all partitions
# this will be very slow for even moderate n and r

r = 3

for i = 1:r
    for j = 1:i
        for p in partitions(i,j)
            println(evalSumNaive(p, Z, D) == evalSumPV(p, Z, D))
        end
    end
end

true
true
true
true
true
true


Ok, looks good! `evalSumPV` is a lot faster than `evalSumNaive`, although they are both very slow. Presumably this is due in part to kludgy coding on my part, but one would imagine that even much better coding practice could only improve these so much

In [5]:
p = [2, 1]
@time evalSumNaive(p, Z, D)
@time evalSumPV(p, Z, D)

  0.085102 seconds (30.42 k allocations: 94.904 MiB, 25.13% gc time)
  0.004549 seconds (2.65 k allocations: 8.912 MiB)


27546

We can use `evalSumsPV` to compute the required sum for each partition. In principle, one could do this with `evalSumsNaive` as well (also implemented), but the latter is VERY slow. 

In [7]:
# need to run twice to avoid timing compile times
r = 5 # size of largest hyperedge
@time MPV = evalSumsPV(Z, D, r)

  1.295019 seconds (504.27 k allocations: 1.791 GiB, 25.13% gc time)


Dict{Any,Any} with 18 entries:
  [2, 2, 1]       => 23050800
  [1, 1, 1, 1]    => 346080
  [3, 2]          => 6288620
  [2, 2]          => 220614
  [3]             => 2750
  [1, 1]          => 1130
  [1, 1, 1]       => 24576
  [2]             => 314
  [2, 1, 1]       => 1175616
  [2, 1]          => 27546
  [4, 1]          => 3587830
  [4]             => 25058
  [1]             => 38
  [5]             => 234638
  [2, 1, 1, 1]    => 26997600
  [3, 1, 1]       => 16723680
  [1, 1, 1, 1, 1] => 2352000
  [3, 1]          => 317768

Remembering that n = 10 in this case and that we will need to evaluate these sums many many times, this timing is not practical. 

Now we'll use the `evalSums`, which uses a recursion lemma from the notes in order to compute the relevant sums for every partition vector p at once. Rather than a simple loop over all partitions like `evalSumsPV` uses, `evalSums` actually uses previously calculated values in order to significantly reduce the per-partition compute time. 

In [10]:
# need to run this block twice in order to avoid timing compile time
@time M = evalSums(Z, D, r)

  0.000191 seconds (1.02 k allocations: 49.359 KiB)


Dict{Array{Integer,1},Integer} with 18 entries:
  Integer[2, 2, 1]       => 23050800
  Integer[1, 1, 1, 1]    => 346080
  Integer[3, 2]          => 6288620
  Integer[2, 2]          => 220614
  Integer[3]             => 2750
  Integer[1, 1]          => 1130
  Integer[1, 1, 1]       => 24576
  Integer[2]             => 314
  Integer[2, 1, 1]       => 1175616
  Integer[2, 1]          => 27546
  Integer[4, 1]          => 3587830
  Integer[4]             => 25058
  Integer[1]             => 38
  Integer[5]             => 234638
  Integer[2, 1, 1, 1]    => 26997600
  Integer[3, 1, 1]       => 16723680
  Integer[1, 1, 1, 1, 1] => 2352000
  Integer[3, 1]          => 317768

The result agrees with `evalSumsPV` on all partitions, but is roughly 10,000 times as fast. 

# How big can we go?

Even with my highly non-optimized code, we can do medium-sized instances with large hyperedges fairly quickly this way. Note, however, that we need BigInts to avoid overflow issues. 

With this method, we can compute on n = 50,000 nodes and hyperedges of up to size 20 in roughly the same compute time that it took `evalSumsPV` to do 10 nodes and hyperedges up to size 5, before further optimization. In a later test (not shown), I was able to compute on 1M nodes in under a minute. 

In [22]:
# Performance test: how big can we do this?
n = 50000

Z = rand(1:50, n)
Z = convert(Array{BigInt,1}, Z)
D = rand(2:100, n)
D = convert(Array{BigInt,1}, D)

r = 20 # maximum hyperedge size

@time M = evalSums(Z, D, r)

  1.663190 seconds (10.40 M allocations: 229.065 MiB, 25.79% gc time)


Dict{Array{Integer,1},Integer} with 2713 entries:
  Integer[11, 6]            => 354679124176778831604771256188372170528107390367…
  Integer[7, 4, 2, 1, 1, 1… => 271451841048001000102990215629915988916793650187…
  Integer[5, 2, 2, 2, 1, 1… => 867994799556550794706126699802340902211063811733…
  Integer[13, 3, 2, 1, 1]   => 651014421678383380535304278242952828530074392436…
  Integer[7, 4, 2, 1]       => 167165023597700996788403334803875164609624313532…
  Integer[6, 1, 1, 1, 1]    => 651049462010418485039859579053412798168896449355…
  Integer[8, 5, 4, 3]       => 292107158136741018386825280338389855149552162278…
  Integer[8, 5, 3, 1, 1, 1… => 264137915180593543203418284905494044535071586145…
  Integer[6, 2, 2, 2, 2]    => 665751776910280533429906274033589323788360533730…
  Integer[6, 3, 1, 1, 1, 1… => 188622219264118905990857674434540909119812466897…
  Integer[8, 4, 2, 2, 1]    => 131420607614019684749821384144174443810512945197…
  Integer[9, 5, 3, 1, 1, 1] => 2692758842686286085618937647